In [16]:
import torch
from torchvision import transforms, datasets
import os
import cv2
import numpy as np
import torch.nn as nn
from torch import optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
DataDir = "/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/B200 Pictures HR"

In [23]:
def CropTrainingData(DataPath):
    
    # Storing Images
    Cropped = []
    
    # Iteration over all0 Image Files
    for Img in tqdm(os.listdir(DataPath)):
        # Reading File Path with Error Handling
        try:
            ImgArray = cv2.imread(os.path.join(DataPath,Img),cv2.IMREAD_GRAYSCALE)
            # Append Image Snippets to Storing Variable
            if ImgArray.shape == (128,128):
                Cropped.append(ImgArray[0:128,0:128])
                Cropped.append(ImgArray[0:128,124:252])
                Cropped.append(ImgArray[0:128,248:376])
                Cropped.append(ImgArray[0:128,372:500])
        except:
            pass
    # convert to Numpy Array and add Dimension like in PyTorch Datasets        
    return np.expand_dims(np.array(Cropped).astype(object),axis=1)

In [24]:
TrainingData = CropTrainingData(DataDir)

100%|██████████| 608/608 [00:01<00:00, 361.78it/s]


In [25]:
print(len(TrainingData))

0


In [ ]:
training_parameters = {
    "n_epochs": 100,
    "batch_size": 100,
}

In [27]:
class GPRDataset(Dataset):
    def __init__ (self, img_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.img_file = img_file
        
    def __len__(self):
        
        return len(img_file)
    
    def __getitem__(self, idx):
        
            
        img_name = os.path.join(self.root_dir, self.img_file[idx])
        image = io.imread(img_name)
        
         #image = image[0:128,0:128]
            
        return image 

if __name__ == '__main__':
    from torch.utils.data import DataLoader
    dataset = GPRDataset(DataDir)
    dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
    for i, batch in enumerate(dataloader):
        print(i, batch)

NameError: name 'Dataset' is not defined